In [157]:
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import pickle
import requests
import re

In [105]:
url = 'https://www.pro-football-reference.com/teams/crd/2019.htm'

response = requests.get(url)

soup = BeautifulSoup(response.text, 'lxml')

In [42]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/pfr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202101052" rel="dns-prefetch"/>
  <title>
   2018 Chicago Bears Statistics &amp; Players | Pro-Football-Reference.com
  </title>
  <meta content="2018 Chicago Bears Statistics &amp; Players: 12-4 (po:0-1) (1st in NFC North), Coach: Matt Nagy, ProBowl: Cohen, Fuller, Hicks, Jackson, Leno, Mack, Trubisky, Whitehair" name="Description"/>
  <link href="https://www.pro-football-reference.com/teams/chi/2018.htm" rel="canonical"/>
  <!-- include:start ="/inc/klecko_header_pfr.html_f" -->
  <!-- no:cookie fast load the css.           -->
  <script>
   function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/,

In [43]:
soup.find_all('table')

[<table class="sortable stats_table" data-cols-to-freeze="1" id="team_stats"><caption>Team Stats and Rankings Table</caption>
 <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
 <thead>
 <tr class="over_header"><th></th><th></th><th></th>
 <th aria-label="" class="over_header center" colspan="3" data-stat="header_tot_off_tgl">Tot Yds &amp; TO</th><th></th><th></th>
 <th aria-label="" class="over_header center" colspan="7" data-stat="header_pass">Passing</th>
 <th aria-label="" class="over_header center" colspan="5" data-stat="header_rush">Rushing</th>
 <th aria-label="" class="over_header center" colspan="3" data-stat="header_penalties">Penalties</th>
 <th aria-label="" class="over_header center" colspan="3" data-stat=""></th>
 <th aria-label="" class="over_header center" colspan="5" data-stat="averages">Average Drive</th>
 </tr>


In [106]:
tables = soup.find('tbody')
rows = tables.find_all('tr')

for row in rows:
    print(row.prettify())


<tr>
 <th class="left" data-stat="player" scope="row">
  Team Stats
 </th>
 <td class="right" data-stat="points">
  361
 </td>
 <td class="right" data-stat="total_yards">
  5467
 </td>
 <td class="right" data-stat="plays_offense">
  1000
 </td>
 <td class="right" data-stat="yds_per_play_offense">
  5.5
 </td>
 <td class="right" data-stat="turnovers">
  18
 </td>
 <td class="right" data-stat="fumbles_lost">
  6
 </td>
 <td class="right" data-stat="first_down">
  314
 </td>
 <td class="right" data-stat="pass_cmp">
  355
 </td>
 <td class="right" data-stat="pass_att">
  554
 </td>
 <td class="right" data-stat="pass_yds">
  3477
 </td>
 <td class="right" data-stat="pass_td">
  20
 </td>
 <td class="right" data-stat="pass_int">
  12
 </td>
 <td class="right" data-stat="pass_net_yds_per_att">
  5.8
 </td>
 <td class="right" data-stat="pass_fd">
  176
 </td>
 <td class="right" data-stat="rush_att">
  396
 </td>
 <td class="right" data-stat="rush_yds">
  1990
 </td>
 <td class="right" data-sta

It looks like all of the data we need for the current team is in the first row. We can get opponent stats from the second row.

In [167]:
team_table = rows[0]
opp_table = rows[1]

team_stats = {}
opp_stats = {}

for row in team_table:
    result = re.findall(r"data-stat=\"([\w]+)\">([\w@]+)", str(row))
    if result != []:
        stat, val = result[0]
        team_stats[stat] = val
        
for row in opp_table:
    result = re.findall(r"data-stat=\"([\w]+)\">([\w]+)", str(row))
    if result != []:
        stat, val = result[0]
        opp_stats[stat] = val
        
# add 'opp' to the beginning of all opponents stats and merge into team_stats dict
for stat in opp_stats.keys():
    new_stat_name = 'opp_' + stat
    opp_stat = opp_stats[stat]
    team_stats[new_stat_name] = opp_stat

team_stats

{'points': '376',
 'total_yards': '5528',
 'plays_offense': '1020',
 'yds_per_play_offense': '5',
 'turnovers': '13',
 'fumbles_lost': '9',
 'first_down': '320',
 'pass_cmp': '356',
 'pass_att': '573',
 'pass_yds': '3733',
 'pass_td': '26',
 'pass_int': '4',
 'pass_net_yds_per_att': '6',
 'pass_fd': '190',
 'rush_att': '411',
 'rush_yds': '1795',
 'rush_td': '18',
 'rush_yds_per_att': '4',
 'rush_fd': '90',
 'penalties': '100',
 'penalties_yds': '774',
 'pen_fd': '40',
 'drives': '177',
 'score_pct': '37',
 'turnover_pct': '6',
 'start_avg': 'Own',
 'time_avg': '2',
 'plays_per_drive': '5',
 'yds_per_drive': '31',
 'points_avg': '2',
 'opp_points': '313',
 'opp_total_yards': '5642',
 'opp_plays_offense': '998',
 'opp_yds_per_play_offense': '5',
 'opp_turnovers': '25',
 'opp_fumbles_lost': '8',
 'opp_first_down': '310',
 'opp_pass_cmp': '326',
 'opp_pass_att': '546',
 'opp_pass_yds': '3721',
 'opp_pass_td': '19',
 'opp_pass_int': '17',
 'opp_pass_net_yds_per_att': '6',
 'opp_pass_fd': '

Great! Now we know how to get stats from this page for all teams. Let's start iterating through multiple teams, storing this data.

We're going to use the same steps we did above, this time in a loop.

In [165]:
team_abbrevs = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den',
                'det', 'gnb', 'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor',
                'nyg', 'nyj', 'rai', 'phi', 'pit', 'sdg', 'sfo', 'sea', 'ram', 'tam',
                'oti', 'was']

['crd',
 'atl',
 'rav',
 'buf',
 'car',
 'chi',
 'cin',
 'cle',
 'dal',
 'den',
 'det',
 'gnb',
 'htx',
 'clt',
 'jax',
 'kan',
 'mia',
 'min',
 'nwe',
 'nor',
 'nyg',
 'nyj',
 'rai',
 'phi',
 'pit',
 'sdg',
 'sfo',
 'sea',
 'ram',
 'tam',
 'oti',
 'was']

In [155]:
def scrape_team_stats(first_year, last_year):
    nfl_team_stats = [] # will contain one dict for each row

    for team in team_abbrevs:

        for year in range(first_year, last_year + 1):

            stats = {} # will contain {stat: value} & be appended to nfl_team_stats

            stats['team'] = team
            stats['year'] = year

            url = f"https://www.pro-football-reference.com/teams/{team}/{year}.htm"
            
            response = requests.get(url)
            if response.status_code == 404:
                continue
            
            print(url) 
            
            soup = BeautifulSoup(response.text, 'lxml')

            table = soup.find('tbody')
            rows = table.find_all('tr')
            team_table = rows[0]
            opp_table = rows[1]

            opp_stats = {}

            for row in team_table:
                result = re.findall(r"data-stat=\"([\w]+)\">([\w]+)", str(row))
                if result != []:
                    stat, val = result[0]
                    stats[stat] = val

            for row in opp_table:
                result = re.findall(r"data-stat=\"([\w]+)\">([\w]+)", str(row))
                if result != []:
                    stat, val = result[0]
                    opp_stats[stat] = val

            # add 'opp' to the beginning of all opponents stats and merge into team_stats dict
            for stat in opp_stats.keys():
                new_stat_name = 'opp_' + stat
                opp_stat = opp_stats[stat]
                stats[new_stat_name] = opp_stat

            nfl_team_stats.append(stats)
        
        
    return nfl_team_stats

nfl_team_stats = scrape_team_stats(1970, 2019)

https://www.pro-football-reference.com/teams/crd/1970.htm
https://www.pro-football-reference.com/teams/crd/1971.htm
https://www.pro-football-reference.com/teams/crd/1972.htm
https://www.pro-football-reference.com/teams/crd/1973.htm
https://www.pro-football-reference.com/teams/crd/1974.htm
https://www.pro-football-reference.com/teams/crd/1975.htm
https://www.pro-football-reference.com/teams/crd/1976.htm
https://www.pro-football-reference.com/teams/crd/1977.htm
https://www.pro-football-reference.com/teams/crd/1978.htm
https://www.pro-football-reference.com/teams/crd/1979.htm
https://www.pro-football-reference.com/teams/crd/1980.htm
https://www.pro-football-reference.com/teams/crd/1981.htm
https://www.pro-football-reference.com/teams/crd/1982.htm
https://www.pro-football-reference.com/teams/crd/1983.htm
https://www.pro-football-reference.com/teams/crd/1984.htm
https://www.pro-football-reference.com/teams/crd/1985.htm
https://www.pro-football-reference.com/teams/crd/1986.htm
https://www.pr

https://www.pro-football-reference.com/teams/buf/1988.htm
https://www.pro-football-reference.com/teams/buf/1989.htm
https://www.pro-football-reference.com/teams/buf/1990.htm
https://www.pro-football-reference.com/teams/buf/1991.htm
https://www.pro-football-reference.com/teams/buf/1992.htm
https://www.pro-football-reference.com/teams/buf/1993.htm
https://www.pro-football-reference.com/teams/buf/1994.htm
https://www.pro-football-reference.com/teams/buf/1995.htm
https://www.pro-football-reference.com/teams/buf/1996.htm
https://www.pro-football-reference.com/teams/buf/1997.htm
https://www.pro-football-reference.com/teams/buf/1998.htm
https://www.pro-football-reference.com/teams/buf/1999.htm
https://www.pro-football-reference.com/teams/buf/2000.htm
https://www.pro-football-reference.com/teams/buf/2001.htm
https://www.pro-football-reference.com/teams/buf/2002.htm
https://www.pro-football-reference.com/teams/buf/2003.htm
https://www.pro-football-reference.com/teams/buf/2004.htm
https://www.pr

https://www.pro-football-reference.com/teams/cin/2005.htm
https://www.pro-football-reference.com/teams/cin/2006.htm
https://www.pro-football-reference.com/teams/cin/2007.htm
https://www.pro-football-reference.com/teams/cin/2008.htm
https://www.pro-football-reference.com/teams/cin/2009.htm
https://www.pro-football-reference.com/teams/cin/2010.htm
https://www.pro-football-reference.com/teams/cin/2011.htm
https://www.pro-football-reference.com/teams/cin/2012.htm
https://www.pro-football-reference.com/teams/cin/2013.htm
https://www.pro-football-reference.com/teams/cin/2014.htm
https://www.pro-football-reference.com/teams/cin/2015.htm
https://www.pro-football-reference.com/teams/cin/2016.htm
https://www.pro-football-reference.com/teams/cin/2017.htm
https://www.pro-football-reference.com/teams/cin/2018.htm
https://www.pro-football-reference.com/teams/cin/2019.htm
https://www.pro-football-reference.com/teams/cle/1970.htm
https://www.pro-football-reference.com/teams/cle/1971.htm
https://www.pr

https://www.pro-football-reference.com/teams/den/2000.htm
https://www.pro-football-reference.com/teams/den/2001.htm
https://www.pro-football-reference.com/teams/den/2002.htm
https://www.pro-football-reference.com/teams/den/2003.htm
https://www.pro-football-reference.com/teams/den/2004.htm
https://www.pro-football-reference.com/teams/den/2005.htm
https://www.pro-football-reference.com/teams/den/2006.htm
https://www.pro-football-reference.com/teams/den/2007.htm
https://www.pro-football-reference.com/teams/den/2008.htm
https://www.pro-football-reference.com/teams/den/2009.htm
https://www.pro-football-reference.com/teams/den/2010.htm
https://www.pro-football-reference.com/teams/den/2011.htm
https://www.pro-football-reference.com/teams/den/2012.htm
https://www.pro-football-reference.com/teams/den/2013.htm
https://www.pro-football-reference.com/teams/den/2014.htm
https://www.pro-football-reference.com/teams/den/2015.htm
https://www.pro-football-reference.com/teams/den/2016.htm
https://www.pr

https://www.pro-football-reference.com/teams/clt/1974.htm
https://www.pro-football-reference.com/teams/clt/1975.htm
https://www.pro-football-reference.com/teams/clt/1976.htm
https://www.pro-football-reference.com/teams/clt/1977.htm
https://www.pro-football-reference.com/teams/clt/1978.htm
https://www.pro-football-reference.com/teams/clt/1979.htm
https://www.pro-football-reference.com/teams/clt/1980.htm
https://www.pro-football-reference.com/teams/clt/1981.htm
https://www.pro-football-reference.com/teams/clt/1982.htm
https://www.pro-football-reference.com/teams/clt/1983.htm
https://www.pro-football-reference.com/teams/clt/1984.htm
https://www.pro-football-reference.com/teams/clt/1985.htm
https://www.pro-football-reference.com/teams/clt/1986.htm
https://www.pro-football-reference.com/teams/clt/1987.htm
https://www.pro-football-reference.com/teams/clt/1988.htm
https://www.pro-football-reference.com/teams/clt/1989.htm
https://www.pro-football-reference.com/teams/clt/1990.htm
https://www.pr

...

https://www.pro-football-reference.com/teams/nor/1984.htm
https://www.pro-football-reference.com/teams/nor/1985.htm
https://www.pro-football-reference.com/teams/nor/1986.htm
https://www.pro-football-reference.com/teams/nor/1987.htm
https://www.pro-football-reference.com/teams/nor/1988.htm
https://www.pro-football-reference.com/teams/nor/1989.htm
https://www.pro-football-reference.com/teams/nor/1990.htm
https://www.pro-football-reference.com/teams/nor/1991.htm
https://www.pro-football-reference.com/teams/nor/1992.htm
https://www.pro-football-reference.com/teams/nor/1993.htm
https://www.pro-football-reference.com/teams/nor/1994.htm
https://www.pro-football-reference.com/teams/nor/1995.htm
https://www.pro-football-reference.com/teams/nor/1996.htm
https://www.pro-football-reference.com/teams/nor/1997.htm
https://www.pro-football-reference.com/teams/nor/1998.htm
https://www.pro-football-reference.com/teams/nor/1999.htm
https://www.pro-football-reference.com/teams/nor/2000.htm
https://www.pr

https://www.pro-football-reference.com/teams/rai/1976.htm
https://www.pro-football-reference.com/teams/rai/1977.htm
https://www.pro-football-reference.com/teams/rai/1978.htm
https://www.pro-football-reference.com/teams/rai/1979.htm
https://www.pro-football-reference.com/teams/rai/1980.htm
https://www.pro-football-reference.com/teams/rai/1981.htm
https://www.pro-football-reference.com/teams/rai/1982.htm
https://www.pro-football-reference.com/teams/rai/1983.htm
https://www.pro-football-reference.com/teams/rai/1984.htm
https://www.pro-football-reference.com/teams/rai/1985.htm
https://www.pro-football-reference.com/teams/rai/1986.htm
https://www.pro-football-reference.com/teams/rai/1987.htm
https://www.pro-football-reference.com/teams/rai/1988.htm
https://www.pro-football-reference.com/teams/rai/1989.htm
https://www.pro-football-reference.com/teams/rai/1990.htm
https://www.pro-football-reference.com/teams/rai/1991.htm
https://www.pro-football-reference.com/teams/rai/1992.htm
https://www.pr

https://www.pro-football-reference.com/teams/pit/2018.htm
https://www.pro-football-reference.com/teams/pit/2019.htm
https://www.pro-football-reference.com/teams/sdg/1970.htm
https://www.pro-football-reference.com/teams/sdg/1971.htm
https://www.pro-football-reference.com/teams/sdg/1972.htm
https://www.pro-football-reference.com/teams/sdg/1973.htm
https://www.pro-football-reference.com/teams/sdg/1974.htm
https://www.pro-football-reference.com/teams/sdg/1975.htm
https://www.pro-football-reference.com/teams/sdg/1976.htm
https://www.pro-football-reference.com/teams/sdg/1977.htm
https://www.pro-football-reference.com/teams/sdg/1978.htm
https://www.pro-football-reference.com/teams/sdg/1979.htm
https://www.pro-football-reference.com/teams/sdg/1980.htm
https://www.pro-football-reference.com/teams/sdg/1981.htm
https://www.pro-football-reference.com/teams/sdg/1982.htm
https://www.pro-football-reference.com/teams/sdg/1983.htm
https://www.pro-football-reference.com/teams/sdg/1984.htm
https://www.pr

https://www.pro-football-reference.com/teams/sea/2017.htm
https://www.pro-football-reference.com/teams/sea/2018.htm
https://www.pro-football-reference.com/teams/sea/2019.htm
https://www.pro-football-reference.com/teams/ram/1970.htm
https://www.pro-football-reference.com/teams/ram/1971.htm
https://www.pro-football-reference.com/teams/ram/1972.htm
https://www.pro-football-reference.com/teams/ram/1973.htm
https://www.pro-football-reference.com/teams/ram/1974.htm
https://www.pro-football-reference.com/teams/ram/1975.htm
https://www.pro-football-reference.com/teams/ram/1976.htm
https://www.pro-football-reference.com/teams/ram/1977.htm
https://www.pro-football-reference.com/teams/ram/1978.htm
https://www.pro-football-reference.com/teams/ram/1979.htm
https://www.pro-football-reference.com/teams/ram/1980.htm
https://www.pro-football-reference.com/teams/ram/1981.htm
https://www.pro-football-reference.com/teams/ram/1982.htm
https://www.pro-football-reference.com/teams/ram/1983.htm
https://www.pr

https://www.pro-football-reference.com/teams/oti/2015.htm
https://www.pro-football-reference.com/teams/oti/2016.htm
https://www.pro-football-reference.com/teams/oti/2017.htm
https://www.pro-football-reference.com/teams/oti/2018.htm
https://www.pro-football-reference.com/teams/oti/2019.htm
https://www.pro-football-reference.com/teams/was/1970.htm
https://www.pro-football-reference.com/teams/was/1971.htm
https://www.pro-football-reference.com/teams/was/1972.htm
https://www.pro-football-reference.com/teams/was/1973.htm
https://www.pro-football-reference.com/teams/was/1974.htm
https://www.pro-football-reference.com/teams/was/1975.htm
https://www.pro-football-reference.com/teams/was/1976.htm
https://www.pro-football-reference.com/teams/was/1977.htm
https://www.pro-football-reference.com/teams/was/1978.htm
https://www.pro-football-reference.com/teams/was/1979.htm
https://www.pro-football-reference.com/teams/was/1980.htm
https://www.pro-football-reference.com/teams/was/1981.htm
https://www.pr

In [3]:
import pandas as pd
nfl_team_stats = [
    {
        
        'fumbles_lost': 17,
        'TD': 40
    },
    {
        
        'fumbles_lost': 23,
        'TD': 46
    },
]

pd.DataFrame(nfl_team_stats)

,fumbles_lost,TD
0,17,40
1,23,46


In [156]:
df = pd.DataFrame(nfl_team_stats)
df

,team,year,points,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,...,yds_per_drive,points_avg,opp_drives,opp_score_pct,opp_turnover_pct,opp_start_avg,opp_time_avg,opp_plays_per_drive,opp_yds_per_drive,opp_points_avg
0,crd,1970,325,4471,845,5,33,14,226,178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,crd,1971,231,4001,821,4,46,20,212,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,crd,1972,193,3267,754,4,39,16,181,171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,crd,1973,286,4054,837,4,29,14,238,210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,crd,1974,285,4314,873,4,17,9,247,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,was,2015,388,5661,1011,5,22,11,317,386,...,31,1,181,35,14,Own,2,5,33,1
1473,was,2016,396,6454,1009,6,21,9,345,407,...,36,2,169,38,11,Own,2,6,35,2
1474,was,2017,342,5199,982,5,27,14,278,347,...,27,1,195,35,11,Own,2,5,28,1
1475,was,2018,281,4795,967,5,19,4,280,311,...,28,1,171,35,14,Own,2,6,33,1


Nice...looks like everything is here. Let's save this file & come back to it in another notebook.

In [160]:
df.to_pickle("data/team_stats_scraped.pickle")

In [163]:
df.team.value_counts()

den    50
buf    50
nyg    50
clt    50
sdg    50
was    50
min    50
ram    50
kan    50
oti    50
dal    50
rai    50
sfo    50
crd    50
atl    50
cin    50
pit    50
chi    50
nwe    50
gnb    50
det    50
nor    50
phi    50
mia    50
nyj    50
cle    47
sea    44
tam    44
car    25
jax    25
rav    24
htx    18
Name: team, dtype: int64

In [166]:
whos

Variable            Type             Data/Info
----------------------------------------------
BeautifulSoup       type             <class 'bs4.BeautifulSoup'>
df                  DataFrame             team  year points to<...>n[1477 rows x 62 columns]
first_data_year     int              2000
i                   Tag              <td class="right" data-st<...>at="points_avg">2.07</td>
last_data_year      int              2019
mask                Series           0      False\n1      Fals<...> Length: 240, dtype: bool
new_stat_name       str              opp_points_avg
nfl_team_stats      list             n=1477
np                  module           <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
opp_stat            str              1
opp_stats           dict             n=30
opp_table           Tag              <tr><th class="left" data<...>oints_avg">1.79</td></tr>
out                 str              data/scraped_team_stats.pickle
pd                  module           <module 'p

In [172]:
df = pd.read_pickle("data/team_stats_scraped.pickle")
mask = (df.team == 'chi') & (df.year == 2000)
df[mask]

,team,year,points,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,...,yds_per_drive,points_avg,opp_drives,opp_score_pct,opp_turnover_pct,opp_start_avg,opp_time_avg,opp_plays_per_drive,opp_yds_per_drive,opp_points_avg
288,chi,2000,216,4541,993,4,29,13,239,304,...,23.0,0.0,195.0,29.0,9.0,Own,2.0,5.0,26.0,1.0
